In [15]:
import pandas as pd
import numpy as np
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE = torch.float64

# True Parameters
init_sigmasq, init_range_lon, init_range_lat = 13.059, 0.195, 0.154 
init_advec_lat, init_range_time, init_advec_lon, init_nugget = 0.0218, 1.0, -0.1689, 0.247

init_phi2 = 1.0 / init_range_lon
init_phi1 = init_sigmasq * init_phi2
init_phi3 = (init_range_lon / init_range_lat)**2
init_phi4 = (init_range_lon / init_range_time)**2

true_params_vec = torch.tensor([
    np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), np.log(init_phi4),
    init_advec_lat, init_advec_lon, np.log(init_nugget)
], device=DEVICE, dtype=DTYPE)

In [16]:
true_params_vec

tensor([ 4.2042,  1.6348,  0.4721, -3.2695,  0.0218, -0.1689, -1.3984],
       dtype=torch.float64)

compare heads 0 vs 100 vs 300

In [ ]:
df0 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_0heads_1217_v050_LBFGS_18126.0.csv")
df100 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_100heads_1217_v050_LBFGS_18126.0.csv")
df300 = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_300heads_1217_v050_LBFGS_18126.0.csv")

df0 = df0.iloc[:,5:(5+7)]
df100 = df100.iloc[:,5:(5+7)]
df300 = df300.iloc[:,5:(5+7)]

# mape

for i in range(7):
    mape0 = np.mean(np.abs((df0.iloc[:,i] - true_params_vec[i].cpu().numpy()) / true_params_vec[i].cpu().numpy())) * 100
    mape100 = np.mean(np.abs((df100.iloc[:,i] - true_params_vec[i].cpu().numpy()) / true_params_vec[i].cpu().numpy())) * 100
    mape300 = np.mean(np.abs((df300.iloc[:,i] - true_params_vec[i].cpu().numpy()) / true_params_vec[i].cpu().numpy())) * 100
    print(f'Parameter {i+1}: MAPE 0 heads: {mape0:.4f}%, MAPE 100 heads: {mape100:.4f}%, MAPE 300 heads: {mape300:.4f}%')



Parameter 1: MAPE 0 heads: 1.2480%, MAPE 100 heads: 1.0542%, MAPE 300 heads: 0.7023%
Parameter 2: MAPE 0 heads: 7.1504%, MAPE 100 heads: 5.4834%, MAPE 300 heads: 2.8699%
Parameter 3: MAPE 0 heads: 2.6854%, MAPE 100 heads: 2.6015%, MAPE 300 heads: 2.4008%
Parameter 4: MAPE 0 heads: 9.9936%, MAPE 100 heads: 10.4038%, MAPE 300 heads: 10.4609%
Parameter 5: MAPE 0 heads: 21.0141%, MAPE 100 heads: 20.9459%, MAPE 300 heads: 20.9470%
Parameter 6: MAPE 0 heads: 4.5566%, MAPE 100 heads: 4.4718%, MAPE 300 heads: 3.9469%
Parameter 7: MAPE 0 heads: 62.0664%, MAPE 100 heads: 49.8431%, MAPE 300 heads: 31.4400%
beta     -0.176092
nugget   -2.266283
dtype: float64
beta     -0.175910
nugget   -2.095356
dtype: float64
beta     -0.174931
nugget   -1.837936
dtype: float64


In [ ]:
print(df0.iloc[:,5:(5+7)].std(), np.mean(df0.iloc[:,5:(5+7)].std()))
print(df100.iloc[:,5:(5+7)].std(), np.mean(df100.iloc[:,5:(5+7)].std()))
print(df300.iloc[:,5:(5+7)].std(), np.mean(df300.iloc[:,5:(5+7)].std()))

sigma        0.015102
range_lat    0.039441
range_lon    0.011914
advec_lat    0.068204
advec_lon    0.005549
beta         0.005588
nugget       0.363741
dtype: float64 0.07279122860034315
sigma        0.015192
range_lat    0.041196
range_lon    0.011941
advec_lat    0.067811
advec_lon    0.005531
beta         0.005578
nugget       0.295666
dtype: float64 0.06327350815109667
sigma        0.014060
range_lat    0.039605
range_lon    0.011910
advec_lat    0.065894
advec_lon    0.005516
beta         0.005144
nugget       0.210278
dtype: float64 0.0503439235857175


compare vecc vs dw

In [ ]:
dw_re = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_dW_v050_121225_18126.0.csv")
vecc_re = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_vecc_1212_v050_LBFGS_18126.0.csv")



#dw_irre = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_irre_dW_v050_121225_18126.0.csv")
#vecc_irre = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_irre_vecc_1212_v050_LBFGS_18126.0.csv")

# 12 17 25:  vecchia used 100 heads and LOC_ERR_STD = 0.018 

dw_irre = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_irre_dW_v050_121725_18126.0.csv")
vecc_irre = pd.read_csv("/Users/joonwonlee/Documents/GEMS_TCO-1/outputs/day/sim_irre_vecc_1217_v050_LBFGS_18126.0.csv")


dw_re = dw_re.iloc[:,5:(5+7)]
dw_irre = dw_irre.iloc[:,5:(5+7)]
vecc_re = vecc_re.iloc[:,5:(5+7)]
vecc_irre = vecc_irre.iloc[:,5:(5+7)]

true_params_vec

tensor([ 4.2042,  1.6348,  0.4721, -3.2695,  0.0218, -0.1689, -1.3984],
       dtype=torch.float64)

In [ ]:
dw_re_list = []
vecc_re_list = []

dw_irre_list = []
vecc_irre_list = []
for i in range(7):
    mape_dw_re = np.mean(np.abs((dw_re.iloc[:,i] - true_params_vec[i].cpu().numpy()) / true_params_vec[i].cpu().numpy())) * 100
    mape_vecc_re = np.mean(np.abs((vecc_re.iloc[:,i] - true_params_vec[i].cpu().numpy()) / true_params_vec[i].cpu().numpy())) * 100
    mape_dw_irre = np.mean(np.abs((dw_irre.iloc[:,i] - true_params_vec[i].cpu().numpy()) / true_params_vec[i].cpu().numpy())) * 100
    mape_vecc_irre = np.mean(np.abs((vecc_irre.iloc[:,i] - true_params_vec[i].cpu().numpy()) / true_params_vec[i].cpu().numpy())) * 100
    dw_re_list.append(mape_dw_re)
    vecc_re_list.append(mape_vecc_re)
    
    dw_irre_list.append(mape_dw_irre)
    vecc_irre_list.append(mape_vecc_irre)
    print(f'Parameter {i+1}: MAPE dw_re: {mape_dw_re:.4f}%, MAPE vecc_re: {mape_vecc_re:.4f}%, MAPE dw_irre: {mape_dw_irre:.4f}%, MAPE vecc_irre: {mape_vecc_irre:.4f}%')


Parameter 1: MAPE dw_re: 0.4254%, MAPE vecc_re: 1.0650%, MAPE dw_irre: 1.6774%, MAPE vecc_irre: 0.4354%
Parameter 2: MAPE dw_re: 2.1808%, MAPE vecc_re: 5.2010%, MAPE dw_irre: 2.3469%, MAPE vecc_irre: 5.7688%
Parameter 3: MAPE dw_re: 3.1790%, MAPE vecc_re: 2.6675%, MAPE dw_irre: 68.4861%, MAPE vecc_irre: 67.9281%
Parameter 4: MAPE dw_re: 1.8318%, MAPE vecc_re: 10.6062%, MAPE dw_irre: 2.4819%, MAPE vecc_irre: 14.7685%
Parameter 5: MAPE dw_re: 29.1983%, MAPE vecc_re: 20.1491%, MAPE dw_irre: 32.2774%, MAPE vecc_irre: 35.4879%
Parameter 6: MAPE dw_re: 4.7754%, MAPE vecc_re: 4.7577%, MAPE dw_irre: 4.7007%, MAPE vecc_irre: 3.7837%
Parameter 7: MAPE dw_re: 13.8970%, MAPE vecc_re: 53.3496%, MAPE dw_irre: 105.9224%, MAPE vecc_irre: 95.8174%


In [54]:
print(dw_irre_list, np.mean(dw_irre_list))
print(vecc_irre_list, np.mean(vecc_irre_list))

print(dw_re_list, np.mean(dw_re_list))
print(vecc_re_list, np.mean(vecc_re_list))

[1.6774477481805259, 2.346939661574694, 68.48605636811395, 2.4819289002677016, 32.27741361848592, 4.70071046369074, 105.92239249394493] 31.127555607751212
[0.4354071215290597, 5.768796410688731, 67.9281099075656, 14.768464162813299, 35.48787536441377, 3.7837263085283848, 95.81738552412702] 31.998537828523695
[0.42538260442180714, 2.1808272403354017, 3.178995588438656, 1.8317566310015279, 29.19834554432457, 4.775401276633183, 13.897023099400736] 7.926818854936554
[1.0650422754094884, 5.200996621379286, 2.667520614402462, 10.606231553873364, 20.149133263797637, 4.757749090653218, 53.34956014357775] 13.970890509013316


In [56]:
print(f'dw_re_std: \n{dw_re.std()}, {np.mean(dw_re.std())}')
print(f'vecc_re_std: \n{vecc_re.std()}, {np.mean(vecc_re.std())}')
print(f'dw_irre_std: \n{dw_irre.std()}, {np.mean(dw_irre.std())}')
print(f'vecc_irre_std: \n{vecc_irre.std()}, {np.mean(vecc_irre.std())}')

dw_re_std: 
sigma        0.021668
range_lat    0.045596
range_lon    0.019254
advec_lat    0.074195
advec_lon    0.007824
beta         0.009693
nugget       0.238422
dtype: float64, 0.05952193928417541
vecc_re_std: 
sigma        0.015237
range_lat    0.035509
range_lon    0.011638
advec_lat    0.059647
advec_lon    0.005502
beta         0.006311
nugget       0.390368
dtype: float64, 0.07488718028016945
dw_irre_std: 
sigma        0.022480
range_lat    0.046544
range_lon    0.024957
advec_lat    0.070531
advec_lon    0.008996
beta         0.010058
nugget       0.043507
dtype: float64, 0.0324390870342526
vecc_irre_std: 
sigma        0.016870
range_lat    0.042611
range_lon    0.013986
advec_lat    0.065940
advec_lon    0.007262
beta         0.006697
nugget       0.038651
dtype: float64, 0.027430911915626195
